In [3]:
from youtube_transcript_api import YouTubeTranscriptApi
import re

from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


In [4]:
# import ssl
# ssl._create_default_https_context = ssl._create_unverified_context

In [5]:
# Add youtube link
youtube_url = "https://www.youtube.com/watch?v=ilFrDBZXJTk"

# extract video ID with regex
video_id_regex = r'(?:v=|\/)([0-9A-Za-z_-]{11}).*'
match = re.search(video_id_regex, youtube_url)

video_id = match.group(1)

transcript_list = YouTubeTranscriptApi.list_transcripts(video_id=video_id)


In [56]:

# iterate over all available transcripts
for transcript in transcript_list:

    # the Transcript object provides metadata properties
    print(
        transcript.video_id,
        transcript.language,
        transcript.language_code,
        # whether it has been manually created or generated by YouTube
        transcript.is_generated,
        # whether this transcript can be translated or not
        transcript.is_translatable,
        # a list of languages the transcript can be translated to
        transcript.translation_languages,
    )

    # fetch the actual transcript data
    print(transcript.fetch())

    # translating the transcript will return another transcript object
    transcript_data = transcript.translate('en').fetch()
    text = "\n".join([item['text'] for item in transcript_data])
    print(text)



ilFrDBZXJTk English (auto-generated) en True True [{'language': 'Abkhazian', 'language_code': 'ab'}, {'language': 'Afar', 'language_code': 'aa'}, {'language': 'Afrikaans', 'language_code': 'af'}, {'language': 'Akan', 'language_code': 'ak'}, {'language': 'Albanian', 'language_code': 'sq'}, {'language': 'Amharic', 'language_code': 'am'}, {'language': 'Arabic', 'language_code': 'ar'}, {'language': 'Armenian', 'language_code': 'hy'}, {'language': 'Assamese', 'language_code': 'as'}, {'language': 'Aymara', 'language_code': 'ay'}, {'language': 'Azerbaijani', 'language_code': 'az'}, {'language': 'Bangla', 'language_code': 'bn'}, {'language': 'Bashkir', 'language_code': 'ba'}, {'language': 'Basque', 'language_code': 'eu'}, {'language': 'Belarusian', 'language_code': 'be'}, {'language': 'Bhojpuri', 'language_code': 'bho'}, {'language': 'Bosnian', 'language_code': 'bs'}, {'language': 'Breton', 'language_code': 'br'}, {'language': 'Bulgarian', 'language_code': 'bg'}, {'language': 'Burmese', 'langu

In [52]:

# you can also directly filter for the language you are looking for, using the transcript list
# transcript = transcript_list.find_transcript(['de', 'en'])  
# print(transcript.fetch())

# or just filter for manually created transcripts  
# transcript = transcript_list.find_manually_created_transcript(['de', 'en'])  
# print(transcript.fetch())

# or automatically generated ones  
# transcript = transcript_list.find_generated_transcript(['de', 'en'])
# print(transcript.translate('en').fetch())


In [53]:
template = """Please provide the summary for the following: {text}"""

full_prompt = ChatPromptTemplate.from_template(template)

In [55]:
# LLM
model = "llama3.1"  # "mistral" | "llama3" | "phi3" | "dolphin-llama3"
llm = ChatOllama(model=model, temperature=0)

# Chain
generate_queries_decomposition = (full_prompt | llm | StrOutputParser() | (lambda x: x.split("\n")))

# Run
summary = generate_queries_decomposition.invoke({"text":text})
summary

["Here's a summary of the text:",
 '',
 'The Large Hadron Collider (LHC) at CERN has led to many fundamental discoveries, including the discovery of the Higgs boson. However, as scientists seek to accelerate particles to even higher energies, they face limitations with traditional accelerators. To overcome this issue, researchers have turned to plasma-based accelerators, which can sustain much larger electric fields than commercial accelerators.',
 '',
 "A team at Laboratoire d'Optique Appliquée (LOA) has successfully accelerated electrons using a laser-plasma accelerator, producing a beam of particles and X-rays similar to those obtained with commercial accelerators. This technology has many potential applications, including:",
 '',
 '* Material science: inspecting dense matter without destroying it',
 '* Medical imaging: producing high-resolution images for cancer diagnosis and treatment',
 '* Radiation therapy: improving cancer treatment by studying the effects of radiation on livin